In [5]:
import torch
from mteb.evaluation.evaluators import RetrievalEvaluator
from colpali_engine.utils.torch_utils import get_torch_device
from mteb.evaluation.evaluators.RetrievalEvaluator import DenseRetrievalExactSearch, DRESModel
from sentence_transformers import SentenceTransformer

class MyEvaluatorWrapper:
    def __init__(self, is_multi_vector=False, retriever_model_name='all-MiniLM-L6-v2'):
        self.is_multi_vector = is_multi_vector
        self.device = get_torch_device()

        # Load a compatible retriever model
        retriever_model = SentenceTransformer(retriever_model_name)
        
        # Define retriever as an instance of DenseRetrievalExactSearch or DRESModel
        retriever = DenseRetrievalExactSearch(DRESModel(retriever_model))
        
        # Initialize RetrievalEvaluator with the retriever
        self.evaluator = RetrievalEvaluator(retriever=retriever)

    def evaluate(self, qs, ps):
        if self.is_multi_vector:
            scores = self.evaluate_colbert(qs, ps)
        else:
            scores = self.evaluate_biencoder(qs, ps)

        assert scores.shape[0] == len(qs)

        arg_score = scores.argmax(dim=1)
        accuracy = (arg_score == torch.arange(scores.shape[0], device=scores.device)).sum().item() / scores.shape[0]
        print(arg_score)
        print(f"Top 1 Accuracy (verif): {accuracy}")

        scores = scores.to(torch.float32).cpu().numpy()
        return scores

    def evaluate_colbert(self, qs, ps, batch_size=128) -> torch.Tensor:
        scores = []
        for i in range(0, len(qs), batch_size):
            scores_batch = []
            qs_batch = torch.nn.utils.rnn.pad_sequence(qs[i : i + batch_size], batch_first=True, padding_value=0).to(
                self.device
            )
            for j in range(0, len(ps), batch_size):
                ps_batch = torch.nn.utils.rnn.pad_sequence(
                    ps[j : j + batch_size], batch_first=True, padding_value=0
                ).to(self.device)
                scores_batch.append(torch.einsum("bnd,csd->bcns", qs_batch, ps_batch).max(dim=3)[0].sum(dim=2))
            scores_batch = torch.cat(scores_batch, dim=1).cpu()
            scores.append(scores_batch)
        scores = torch.cat(scores, dim=0)
        return scores

    def evaluate_biencoder(self, qs, ps) -> torch.Tensor:
        qs = torch.stack(qs)
        ps = torch.stack(ps)
        scores = torch.einsum("bd,cd->bc", qs, ps)
        return scores


In [6]:
import gradio as gr
import torch
from dotenv import load_dotenv
import os
from PIL import Image
from typing import Tuple, List
import google.generativeai as genai
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.utils.colpali_processing_utils import process_queries, process_images
from transformers import AutoProcessor, BitsAndBytesConfig
from pdf2image import convert_from_path
from torch.utils.data import DataLoader

# Load the API key from the .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=gemini_api_key)

# Model configuration
model_name = "vidore/colpali"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)

# Load RAG model
retrieval_model = ColPali.from_pretrained("google/paligemma-3b-mix-448",
                                          torch_dtype=torch.float16,
                                          device_map="cuda",
                                          quantization_config=bnb_config).eval()
retrieval_model.load_adapter(model_name)
paligemma_processor = AutoProcessor.from_pretrained(model_name)
device = retrieval_model.device

# Define index function to process PDFs
def index(files: List[str]) -> Tuple[List[torch.Tensor], List[Image.Image]]:
    poppler_path = r"C:\Poppler\poppler-24.08.0\Library\bin"
    images = []
    document_embeddings = []

    # Convert PDF pages to images
    for file in files:
        print(f"Indexing now: {file}")
        images.extend(convert_from_path(file, poppler_path=poppler_path))

    # Create DataLoader for images
    dataloader = DataLoader(
        images,
        batch_size=1,
        shuffle=False,
        collate_fn=lambda x: process_images(paligemma_processor, x),
    )

    # Generate embeddings for each image batch
    for batch in dataloader:
        with torch.no_grad():
            batch = {key: value.to(device) for key, value in batch.items()}
            embeddings = retrieval_model(**batch)
        document_embeddings.extend(list(torch.unbind(embeddings.to("cpu"))))
    
    return document_embeddings, images

# Load document files and index them
DATA_FOLDER = "C:\\Users\\gargm\\Desktop\\Projects\\BTech\\Raw_DataFiles"
pdf_files = [os.path.join(DATA_FOLDER, file) for file in os.listdir(DATA_FOLDER) if file.lower().endswith('.pdf')]
document_embeddings, images = index(pdf_files)

# Function to retrieve the top document
def retrieve_top_document(query: str, document_embeddings: List[torch.Tensor], document_images: List[Image.Image]) -> Tuple[Image.Image, int]:
    placeholder_image = Image.new("RGB", (448, 448), (255, 255, 255))

    with torch.no_grad():
        query_batch = process_queries(paligemma_processor, [query], placeholder_image)
        query_batch = {key: value.to(device) for key, value in query_batch.items()}
        query_embeddings_tensor = retrieval_model(**query_batch)
        query_embeddings = list(torch.unbind(query_embeddings_tensor.to("cpu")))

    retriever_evaluator = MyEvaluatorWrapper(is_multi_vector=True)
    similarity_scores = retriever_evaluator.evaluate(query_embeddings, document_embeddings)
    best_index = int(similarity_scores.argmax(axis=1).item())
    
    return document_images[best_index], best_index

# Gemini answer generation
def get_answer(prompt: str, image: Image) -> str:
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content([prompt, image])
    return response.text

# Main function to handle query and prompt
def answer_query(query: str, prompt: str) -> Tuple[str, Image.Image, int]:
    best_image, best_index = retrieve_top_document(query, document_embeddings, images)
    answer = f"Gemini Response:\n {get_answer(prompt, best_image)}"
    return answer, best_image, best_index

# Gradio response function
def gradio_response(query, prompt):
    answer, image, index = answer_query(query, prompt)
    return answer, image

# Define Gradio Interface
def chatbot_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Chatbot Name - RAG & Gemini Integration")

        with gr.Row():
            with gr.Column(scale=3):
                query_box = gr.Textbox(label="Search Query", placeholder="Enter search query", lines=1)
                prompt_box = gr.Textbox(label="Question", placeholder="Enter question for the document", lines=2)
                submit_button = gr.Button("Submit")
                answer_output = gr.Textbox(label="Response")
                image_output = gr.Image(label="Retrieved Image")

                # Link Gradio inputs and outputs to the function
                submit_button.click(
                    fn=gradio_response,
                    inputs=[query_box, prompt_box],
                    outputs=[answer_output, image_output]
                )

        demo.launch()

chatbot_interface()


Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.76s/it]
Some weights of ColPali were not initialized from the model checkpoint at google/paligemma-3b-mix-448 and are newly initialized: ['custom_text_proj.bias', 'custom_text_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\181930-Prompt-Milk-Analyzer-Brochure-12th-November-2019-for-web-and-mobile-only.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\A5-8.3_x_5.8_inch_ThawEasy_Lite.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\A5-8.3_x_5.8_inch_ThawEasy_Lite_Plus.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\A5-8.3_x_5.8_inch_ThawEasy_Smart.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\A5-8.3_x_5.8_inch_ThawEasy_Smart_Pro.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\AMCS-brochure-v2.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\AMCS-Brochure.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\AMCS_Client_Software_-_User_Manual.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\Amul-Logisitic-App.pdf
Indexing now: C:\Users\gargm\Desktop\Projects\BTech\Raw_DataFiles\

c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\transformers\models\siglip\modeling_siglip.py:574: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([1404])
Top 1 Accuracy (verif): 0.0


Traceback (most recent call last):
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\google\api_core\grpc_helpers.py", line 76, in error_remapped_callable
    return callable_(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\grpc\_channel.py", line 1181, in __call__
    return _end_unary_response_blocking(state, call, False, None)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\grpc\_channel.py", line 1006, in _end_unary_response_blocking
    raise _InactiveRpcError(state)  # pytype: disable=not-instantiable
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:142.250.194.42:443: tcp handshaker shutdown"
	debug_error_string = "UNKNOWN:Err

tensor([1404])
Top 1 Accuracy (verif): 0.0


Traceback (most recent call last):
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\google\api_core\grpc_helpers.py", line 76, in error_remapped_callable
    return callable_(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\grpc\_channel.py", line 1181, in __call__
    return _end_unary_response_blocking(state, call, False, None)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\grpc\_channel.py", line 1006, in _end_unary_response_blocking
    raise _InactiveRpcError(state)  # pytype: disable=not-instantiable
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:142.250.194.42:443: tcp handshaker shutdown"
	debug_error_string = "UNKNOWN:Err

In [8]:
import os
import torch
from PIL import Image
from dotenv import load_dotenv
import gradio as gr
from pdf2image import convert_from_path
from typing import List, Tuple
from transformers import AutoProcessor, BitsAndBytesConfig
import google.generativeai as genai
from torch.utils.data import DataLoader
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.utils.colpali_processing_utils import process_images, process_queries
from sentence_transformers import SentenceTransformer
from mteb.evaluation.evaluators import RetrievalEvaluator
from mteb.evaluation.evaluators.RetrievalEvaluator import DenseRetrievalExactSearch, DRESModel

# Load API key from .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=gemini_api_key)

# Model configuration
model_name = "vidore/colpali"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

# Load RAG model and processor
retrieval_model = ColPali.from_pretrained("google/paligemma-3b-mix-448", torch_dtype=torch.float16, 
                                          device_map="cuda", quantization_config=bnb_config).eval()
retrieval_model.load_adapter(model_name)
paligemma_processor = AutoProcessor.from_pretrained(model_name)
device = retrieval_model.device

# Function to process and cache embeddings and images efficiently
def index_optimized(files: List[str], embedding_cache: str = 'document_embeddings_cache.pt', image_dir: str = 'cached_images') -> Tuple[List[torch.Tensor], List[str]]:
    # Check for cached embeddings and images
    if os.path.exists(embedding_cache) and os.path.isdir(image_dir):
        document_embeddings = torch.load(embedding_cache)
        image_paths = [os.path.join(image_dir, img) for img in sorted(os.listdir(image_dir))]
        return document_embeddings, image_paths

    # Initialize lists for embeddings and create the image directory
    os.makedirs(image_dir, exist_ok=True)
    document_embeddings = []
    poppler_path = r"C:\Poppler\poppler-24.08.0\Library\bin"  # Adjust this path as needed
    image_paths = []
    
    # Convert PDF pages to images and save each image separately
    for file in files:
        images = convert_from_path(file, poppler_path=poppler_path)
        for i, image in enumerate(images):
            image_path = os.path.join(image_dir, f"{os.path.basename(file)}_page_{i}.png")
            image.save(image_path, "PNG")
            image_paths.append(image_path)

    # Process images in batches for embedding generation
    dataloader = DataLoader([Image.open(p) for p in image_paths], batch_size=4, shuffle=False, collate_fn=lambda x: process_images(paligemma_processor, x))
    for batch in dataloader:
        with torch.no_grad():
            batch = {key: value.to(device) for key, value in batch.items()}
            embeddings = retrieval_model(**batch).to("cpu").float()  # Convert to float32 for compatibility
            document_embeddings.extend(list(torch.unbind(embeddings)))

    # Save embeddings to cache
    torch.save(document_embeddings, embedding_cache)
    return document_embeddings, image_paths

# Load PDF files and index them
DATA_FOLDER = "C:\\Users\\gargm\\Desktop\\Projects\\BTech\\Raw_DataFiles"
pdf_files = [os.path.join(DATA_FOLDER, file) for file in os.listdir(DATA_FOLDER) if file.lower().endswith('.pdf')]
document_embeddings, image_paths = index_optimized(pdf_files)

# Wrapper for evaluating similarity
class MyEvaluatorWrapper:
    def __init__(self, retriever_model_name='all-MiniLM-L6-v2'):
        self.device = device
        retriever_model = SentenceTransformer(retriever_model_name)
        retriever = DenseRetrievalExactSearch(DRESModel(retriever_model))
        self.evaluator = RetrievalEvaluator(retriever=retriever)

    def evaluate(self, qs, ps):
        # Flatten embeddings to ensure they are 2D
        qs = [q.view(-1, q.shape[-1]).mean(dim=0) if q.dim() > 1 else q for q in qs]
        ps = [p.view(-1, p.shape[-1]).mean(dim=0) if p.dim() > 1 else p for p in ps]

        # Stack the embeddings and convert them to float32 for compatibility
        qs, ps = torch.stack(qs).to(torch.float32), torch.stack(ps).to(torch.float32)
        
        # Ensure both tensors are 2D before applying einsum
        assert qs.dim() == 2 and ps.dim() == 2, f"Expected 2D tensors but got {qs.dim()}D for qs and {ps.dim()}D for ps"
        
        # Compute similarity scores
        return torch.einsum("bd,cd->bc", qs, ps)

# Retrieve the best matching document
def retrieve_top_document(query: str, document_embeddings: List[torch.Tensor]) -> Tuple[str, int]:
    placeholder_image = Image.new("RGB", (448, 448), (255, 255, 255))  # Placeholder image for query
    with torch.no_grad():
        query_batch = process_images(paligemma_processor, [placeholder_image])
        query_embeddings_tensor = retrieval_model(**{k: v.to(device) for k, v in query_batch.items()}).to("cpu").float()
        query_embeddings = list(torch.unbind(query_embeddings_tensor))

    # Retrieve the document most similar to the query
    retriever_evaluator = MyEvaluatorWrapper()
    similarity_scores = retriever_evaluator.evaluate(query_embeddings, document_embeddings)
    best_index = int(similarity_scores.argmax(dim=1).item())
    return image_paths[best_index], best_index

# Generate answer using Gemini API
def get_answer(prompt: str, image_path: str) -> str:
    image = Image.open(image_path)
    response = genai.GenerativeModel(model_name="gemini-1.5-flash").generate_content([prompt, image])
    return response.text

# Main query function
def answer_query(query: str, prompt: str) -> Tuple[str, str]:
    best_image_path, _ = retrieve_top_document(query, document_embeddings)
    answer = f"Generated Response:\n{get_answer(prompt, best_image_path)}"
    return answer, best_image_path

# Gradio response function
def gradio_response(query, prompt):
    return answer_query(query, prompt)

# Gradio Interface
def chatbot_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Chatbot")
        with gr.Row():
            query_box = gr.Textbox(label="Search Query", placeholder="Enter the product name: ", lines=1)
            prompt_box = gr.Textbox(label="Question", placeholder="What is the issue? ", lines=2)
            answer_output = gr.Textbox(label="Response")
            image_output = gr.Image(label="Retrieved Image")
            submit_button = gr.Button("Submit")
            submit_button.click(fn=gradio_response, inputs=[query_box, prompt_box], outputs=[answer_output, image_output])
        demo.launch()

# Launch Gradio interface
chatbot_interface()


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.13s/it]
Some weights of ColPali were not initialized from the model checkpoint at google/paligemma-3b-mix-448 and are newly initialized: ['custom_text_proj.bias', 'custom_text_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import os
import torch
from PIL import Image
from dotenv import load_dotenv
import gradio as gr
from pdf2image import convert_from_path
from typing import List, Tuple
from transformers import AutoProcessor, BitsAndBytesConfig
import google.generativeai as genai
from torch.utils.data import DataLoader
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.utils.colpali_processing_utils import process_images
from sentence_transformers import SentenceTransformer
from mteb.evaluation.evaluators import RetrievalEvaluator
from mteb.evaluation.evaluators.RetrievalEvaluator import DenseRetrievalExactSearch, DRESModel
import tempfile  # To handle temporary image file saving

# Load API key from .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Configure Gemini API
genai.configure(api_key=gemini_api_key)

# Model configuration
model_name = "vidore/colpali"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

# Load RAG model and processor
retrieval_model = ColPali.from_pretrained("google/paligemma-3b-mix-448", torch_dtype=torch.float16, 
                                          device_map="cuda", quantization_config=bnb_config).eval()
retrieval_model.load_adapter(model_name)
paligemma_processor = AutoProcessor.from_pretrained(model_name)
device = retrieval_model.device

# Function for loading or generating embeddings
def load_or_generate_embeddings(files: List[str], embedding_cache: str = 'document_embeddings_cache.pt', image_dir: str = 'cached_images') -> Tuple[List[torch.Tensor], List[str]]:
    if os.path.exists(embedding_cache) and os.path.isdir(image_dir):
        document_embeddings = torch.load(embedding_cache)
        image_paths = [os.path.join(image_dir, img) for img in sorted(os.listdir(image_dir))]
        return document_embeddings, image_paths

    os.makedirs(image_dir, exist_ok=True)
    document_embeddings = []
    poppler_path = r"C:\Poppler\poppler-24.08.0\Library\bin"
    image_paths = []
    
    for file in files:
        images = convert_from_path(file, poppler_path=poppler_path)
        for i, image in enumerate(images):
            image_path = os.path.join(image_dir, f"{os.path.basename(file)}_page_{i}.png")
            image.save(image_path, "PNG")
            image_paths.append(image_path)

    dataloader = DataLoader([Image.open(p) for p in image_paths], batch_size=1, shuffle=False, collate_fn=lambda x: process_images(paligemma_processor, x))
    for batch in dataloader:
        with torch.no_grad():
            batch = {key: value.to(device) for key, value in batch.items()}
            embeddings = retrieval_model(**batch).to("cpu").float()
            document_embeddings.extend(list(torch.unbind(embeddings)))

    torch.save(document_embeddings, embedding_cache)
    return document_embeddings, image_paths

# Load PDF files and index them
DATA_FOLDER = "C:\\Users\\gargm\\Desktop\\Projects\\BTech\\Raw_DataFiles"
pdf_files = [os.path.join(DATA_FOLDER, file) for file in os.listdir(DATA_FOLDER) if file.lower().endswith('.pdf')]
document_embeddings, image_paths = load_or_generate_embeddings(pdf_files)

# Wrapper for evaluating similarity
class MyEvaluatorWrapper:
    def __init__(self, retriever_model_name='all-MiniLM-L6-v2'):
        self.device = device
        retriever_model = SentenceTransformer(retriever_model_name)
        retriever = DenseRetrievalExactSearch(DRESModel(retriever_model))
        self.evaluator = RetrievalEvaluator(retriever=retriever)

    def evaluate(self, qs, ps):
        qs = [q.view(-1, q.shape[-1]).mean(dim=0) if q.dim() > 1 else q for q in qs]
        ps = [p.view(-1, p.shape[-1]).mean(dim=0) if p.dim() > 1 else p for p in ps]
        qs, ps = torch.stack(qs).to(torch.float32), torch.stack(ps).to(torch.float32)
        return torch.einsum("bd,cd->bc", qs, ps)

# Retrieve the best matching document
def retrieve_top_document(query: str, document_embeddings: List[torch.Tensor]) -> Tuple[str, int]:
    placeholder_image = Image.new("RGB", (448, 448), (255, 255, 255))
    with torch.no_grad():
        query_batch = process_images(paligemma_processor, [placeholder_image])
        query_embeddings_tensor = retrieval_model(**{k: v.to(device) for k, v in query_batch.items()})
        query_embeddings = list(torch.unbind(query_embeddings_tensor.to("cpu")))

    retriever_evaluator = MyEvaluatorWrapper()
    similarity_scores = retriever_evaluator.evaluate(query_embeddings, document_embeddings)
    best_index = int(similarity_scores.argmax(dim=1).item())
    return image_paths[best_index], best_index

# Generate answer using Gemini API
def get_answer(prompt: str, image_path: str) -> str:
    image = Image.open(image_path)
    response = genai.GenerativeModel(model_name="gemini-1.5-flash").generate_content([prompt, image])
    return response.text

# Main query function
def answer_query(query: str, prompt: str) -> Tuple[str, str]:
    best_image_path, _ = retrieve_top_document(query, document_embeddings)
    answer = get_answer(prompt, best_image_path)
    return answer, best_image_path

# Gradio response function for chat-style output
def gradio_response(history, user_input):
    query, prompt = history[-1] if history else ("", user_input)
    answer, image_path = answer_query(query, prompt)

    # Save the retrieved image to a temporary file
    with Image.open(image_path) as img:
        temp_img_path = tempfile.NamedTemporaryFile(suffix=".png", delete=False).name
        img.save(temp_img_path)

    # Append both text and image path to the chat history
    history.append((user_input, (answer, temp_img_path)))
    return history, ""

# Gradio Interface with custom styling
def chatbot_interface():
    with gr.Blocks(css="""
        body {background-color: #2a2866;}
        .gradio-container {background-color: #2a2866;}
        #chatbot {background-color: #c7e6f8; padding: 10px; border-radius: 8px;}
        #chatbot .message {
            background-color: #808080;
            color: white;
            border-radius: 5px;
            padding: 8px;
            margin: 5px 0;
        }
        #input_box textarea {
            background-color: #c7e6f8;
            color: black;
            border: none;
            border-radius: 5px;
            padding: 8px;
        }
        .gr-button.new-chat {
            color: white;
            border: 1px solid white;
            padding: 8px;
            background: none;
        }
        .history-button {
            background: none;
            border: none;
            color: white;
            font-size: 14px;
            text-align: left;
            display: flex;
            align-items: center;
            gap: 8px;
            cursor: pointer;
            justify-content: flex-start;
        }
        .history-button::before {
            content: "\\1F5E8";
            margin-right: 5px;
        }
        .history-button:hover {
            color: #c7e6f8;
        }
        #send_button {
            width: 36px;
            height: 36px;
            background-color: #c7e6f8;
            color: black;
            border: none;
            font-size: 16px;
            display: flex;
            align-items: center;
            justify-content: center;
            border-radius: 5px;
        }
    """) as demo:
        with gr.Row():
            gr.Markdown("# Chatbot Interface", elem_id="chatbot-title")

        with gr.Row():
            with gr.Column(scale=0.7):
                gr.Button("+ New chat", elem_classes=["new-chat"])
                for i in range(3):
                    gr.Button(f"Chat {i+1}", elem_classes=["history-button"])

            with gr.Column(scale=3.3):
                chatbot = gr.Chatbot(elem_id="chatbot")
                with gr.Row():
                    input_box = gr.Textbox(show_label=False, placeholder="Type your question here", lines=1, elem_id="input_box")
                    send_button = gr.Button("\u27A4", elem_id="send_button")

                    send_button.click(fn=gradio_response, inputs=[chatbot, input_box], outputs=[chatbot, input_box])
                    input_box.submit(fn=gradio_response, inputs=[chatbot, input_box], outputs=[chatbot, input_box])

    demo.launch()

# Launch Gradio interface
chatbot_interface()


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.69s/it]
Some weights of ColPali were not initialized from the model checkpoint at google/paligemma-3b-mix-448 and are newly initialized: ['custom_text_proj.bias', 'custom_text_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.7 will cause issues.
  warnings.warn(
c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 3.3 will cause issues.
  warnings.warn(
c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be remov

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gargm\Desktop\Projects\BTech\.venv\Lib\site-packages\gradio\blocks.py", line 2028, in proces